In [1]:
import torch
import os

device = torch.device("cuda:0") 

In [2]:
torch.cuda.is_available()

True

In [3]:
!ls

CLDR.ipynb		   config_gpu.sh	     Preprocessing.ipynb
config_cpu_pyScript_MP.sh  doc2vec_embeddings.ipynb  __pycache__
config_cpu_pyScript.sh	   embeddings.py	     Thesis
config_cpu.sh		   mapping_methods.py	     visualisation.ipynb
config_gpu_pyScript_MP.sh  mapping_script_MP.py
config_gpu_pyScript.sh	   mapping_script.py


In [4]:
!mkdir Thesis
%cd Thesis

mkdir: cannot create directory ‘Thesis’: File exists
/home2/s4231317/Thesis_All/Thesis


In [5]:
languages = ["en", "ro", "fr", "de", "nl"]

if "Data" not in os.listdir(): # first generate the data locally
  !mkdir Data

  %cd Data

  for lang in languages:
      if lang in os.listdir():
        print(f"{lang} is already present")
        continue
      link = f"https://wt-public.emm4u.eu/Acquis/JRC-Acquis.3.0/corpus/jrc-{lang}.tgz"
      !wget $link
      file_name = f"jrc-{lang}.tgz"
      !tar xzf $file_name
      !rm jrc*
      %cd $lang
      path = f"jrc-{lang}.xml"
      !rm $path
      path = f"jrc-{lang}.xml~"
      !rm $path
      path = f"jrcHeader-{lang}.html"
      !rm $path
      path = f"jrcHeader-{lang}.xml"
      !rm $path
      path = f"tei2-ext.dtd"
      !rm $path
      path = f"tei2.dtd"
      !rm $path
      %cd ..
else:
  %cd Data
  print("All data are downloaded")

/home2/s4231317/Thesis_All/Thesis/Data
All data are downloaded


In [6]:
import re
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import ElementTree
from html.entities import html5
import pandas as pd
from tqdm import tqdm

def read_convert_whole(path, lang):
  c=[]
  for folder in os.listdir(path):
    for filename in tqdm(os.listdir(path+"/"+folder), desc=f"{lang}-{folder}"):
        tree = ET.parse(path+"/"+folder+"/"+filename)
        root = tree.getroot()
        number = root.attrib["n"]
        body = ""
        
        for text in root[1].itertext():
          text = re.sub(r"[\r\n\t\f\v]+", "", text) # remove whitespace characters except " "
          text = re.sub(r"[ ]{2,}", " ", text) # reduce consecutive spaces to singular spaces
            
          bad_format = re.findall(r"(%[a-zA-z]+%)", text) # e.g. %eacute%
          if len(bad_format) > 1:
            for symbol in bad_format:
              symbol = symbol[1:-1] + ";"
              if symbol in html5.keys():
                text = re.sub(r"(%[a-zA-z]+%)", html5[symbol], text, 1)
                
          body += text
        c.append([number, body]) #,title,body])
  df = pd.DataFrame (c,columns=['number', f"body_{lang}"]) #,'title','body']) 
  return df

In [7]:
!ls

data_merged.csv  de  en  fr  nl  ro


In [8]:
from functools import reduce

if "data_merged.csv" not in os.listdir():
  dfs = []
  cols = ["number"]
  for lang in languages:
      dfs.append(read_convert_whole(lang, lang))
      cols.append(f"body_{lang}")

  df_merged = reduce(lambda left,right:pd.merge(left,right, on=["number"]), dfs)

  df_merged.to_csv("data_merged.csv", index=False)
else:
  print("Data were downloaded - loading")
  df_merged = pd.read_csv("data_merged.csv")

Data were downloaded - loading


In [9]:
df_merged["body_de"][0]

'  Verordnung (EWG) Nr. 2264/69 der Kommission vom 13. November 1969 über die Anträge auf Rückvergütung der den 0rganisationen von Obst- und Gemüseerzeugern von den Mitgliedstaaten gewährten Beihilfen  VERORDNUNG (EWG) Nr. 2264/69 DER KOMMISSION vom 13. November 1969 über die Anträge auf Rückvergütung der den Organisationen von Obst- und Gemüseerzeugern von den Mitgliedstaaten gewährten Beihilfen DIE KOMMISSION DER EUROPÄISCHEN GEMEINSCHAFTEN - gestützt auf den Vertrag zur Gründung der Europäischen Wirtschaftsgemeinschaft, gestützt auf die Verordnung Nr. 159/66/EWG des Rates vom 25. Oktober 1966 mit zusätzlichen Vorschriften für die gemeinsame Marktorganisation für Obst und Gemüse (1), gestützt auf die Verordnung (EWG) Nr. 499/69 des Rates vom 11. März 1969 über die Rückvergütung der den Organisationen von Obst- und Gemüseerzeugern von den Mitgliedstaaten gewährten Beihilfen (2), insbesondere auf Artikel 7 Absatz 3, und in Erwägung nachstehender Gründe: Die Anträge auf Rückvergütung de

In [10]:
df_merged.head()

,number,body_en,body_ro,body_fr,body_de,body_nl
0,31969R2264,Regulation (EEC) No 2264/69 of the Commissio...,REGULAMENTUL COMISIEI (CEE) nr. 2264/69 din...,"Règlement (CEE) n° 2264/69 de la Commission,...",Verordnung (EWG) Nr. 2264/69 der Kommission ...,Verordening (EEG) nr. 2264/69 van de Commiss...
1,31969L0466,Council Directive 69/466/EEC of 8 December 1...,DIRECTIVA 69/466/CEE A CONSILIULUI din 8 de...,"Directive 69/466/CEE du Conseil, du 8 décemb...",Richtlinie 69/466/EWG des Rates vom 8. Dezem...,Richtlijn 69/466/EEG van de Raad van 8 decem...
2,31969R2602,Regulation (EEC) No 2602/69 of the Council o...,REGULAMENTUL (CEE) nr. 2602/69 AL CONSILIUL...,"Règlement (CEE) n° 2602/69 du Conseil, du 18...",Verordnung (EWG) Nr. 2602/69 des Rates vom 1...,Verordening (EEG) nr. 2602/69 van de Raad va...
3,31969L0464,Council Directive 69/464/EEC of 8 December 1...,DIRECTIVA 69/464/CEE A CONSILIULUI din 8 de...,"Directive 69/464/CEE du Conseil, du 8 décemb...",Richtlinie 69/464/EWG des Rates vom 8. Dezem...,Richtlijn 68/464/EEG van de Raad van 8 decem...
4,31969L0063,Council Directive 69/63/EEC of 18 February 1...,DIRECTIVA CONSILIULUI din 18 februarie 1969...,"Directive 69/63/CEE du Conseil, du 18 févrie...",Richtlinie 69/63/EWG des Rates vom 18. Febru...,Richtlijn 69/63/EEG van de Raad van 18 febru...


In [11]:
df_merged.shape

(6538, 6)

In [12]:
import torch

device = torch.device("cuda:0") 

model_names = ["bert-base-multilingual-uncased", "xlm-roberta-base", "google/mt5-base", "susnato/ernie-m-base_pytorch"]

%cd ~/Thesis_All/Thesis

if "Models" not in os.listdir():
  !mkdir Models

%cd Models  
!git lfs install
for model_name in model_names:
  aux = model_name.split("/")
  aux = aux[1] if len(aux) > 1 else aux[0]
  if aux in os.listdir():
    print(f"{aux} has already been downloaded")
    continue
  !git clone https://huggingface.co/$model_name
    
%cd ..


/home2/s4231317/Thesis_All/Thesis
/home2/s4231317/Thesis_All/Thesis/Models
Updated Git hooks.
Git LFS initialized.
bert-base-multilingual-uncased has already been downloaded
xlm-roberta-base has already been downloaded
mt5-base has already been downloaded
ernie-m-base_pytorch has already been downloaded
/home2/s4231317/Thesis_All/Thesis


In [13]:
!ls

Data  Models


In [14]:
models = ["bert-base-multilingual-uncased", "mt5-base", "xlm-roberta-base", "ernie-m-base_pytorch"]

In [19]:
import torch.multiprocessing as mp
from embeddings import get_embeddings

def get_embeddings_loop(lang, data, folder_name): 
  processes = []
  path = f"Embeddings/{folder_name}/"
  for model_name in models:
      if f"emb_{model_name}_{lang}.npy" in os.listdir(path):
        print(f"{model_name} for {lang} was already used to create embeddings, going to next model")
        continue

      p = mp.Process(target=get_embeddings, args=(data, lang, model_name, path))
      p.start()
      processes.append(p)

  for p in processes:
    p.join()

In [20]:
!ls

Data  Embeddings  Models


In [21]:
sample_max = 7000

%cd ~/Thesis

!mkdir Embeddings

if __name__ == "__main__":
  for lang in languages:
    mp.set_start_method('spawn', force=True)
    folder_name = f"{lang}"
    !mkdir Embeddings/$folder_name

    get_embeddings_loop(lang, df_merged[f"body_{lang}"][:sample_max], folder_name)

[Errno 2] No such file or directory: '/home2/s4231317/Thesis'
/home2/s4231317/Thesis_All/Thesis
mkdir: cannot create directory ‘Embeddings’: File exists
mkdir: cannot create directory ‘Embeddings/en’: File exists


Some weights of the model checkpoint at Models/bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using BertModel with BertTokenizer


bert-base-multilingual-uncased - en:   1%|▏         | 91/6538 [00:08<06:21, 16.90it/s]Some weights of the model checkpoint at Models/xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
bert-base-multilingual-uncased - en:   2%|▏         | 110/6538 [00:09<05:17, 20.25it/s]

Using XLMRobertaModel with XLMRobertaTokenizer


...

mt5-base - nl:   0%|          | 13/6538 [00:01<07:32, 14.43it/s] :05<10:13, 10.56it/s]

Using XLMRobertaModel with XLMRobertaTokenizer


mt5-base - nl:   1%|          | 69/6538 [00:03<05:13, 20.60it/s]/s]it/s]07, 17.57it/s]

Using ErnieMModel with ErnieMTokenizer


bert-base-multilingual-uncased - nl: 100%|██████████| 6538/6538 [08:00<00:00, 13.61it/s]
